In [ ]:
# WARNING: advised to install a specific version, e.g. tensorwaves==0.1.2
%pip install -q tensorwaves[doc,jax,pwa,viz] IPython

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

from IPython.display import display  # noqa: F401

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Core ideas illustrated

At core, {mod}`tensorwaves` is a package that can 'fit' arbitrary mathematical expressions to a data set using different computational back-ends. It can also use those expressions to describe a distribution over which to generate data samples.

This page illustrate what's going on behind the scenes with some simple 1-dimensional and 2-dimensional expressions. The main steps are:

1. Formulate a mathematical expression with {mod}`sympy`.
2. Generate a distribution data sample for that expression.
3. Express the expression as a function in some computational back-end.
4. Tweak the {attr}`~.ParametrizedFunction.parameters` and fit the {class}`.ParametrizedFunction` to the generated distribution.

In [ ]:
%pip -q install black

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sp
from sympy.plotting import plot3d

from tensorwaves.estimator import UnbinnedNLL
from tensorwaves.function.sympy import create_parametrized_function
from tensorwaves.optimizer import Minuit2, ScipyMinimizer
from tensorwaves.optimizer.callbacks import (
    CallbackList,
    CSVSummary,
    TFSummary,
    YAMLSummary,
)

## Formulate model

First of all, we'll formulate some expression with {mod}`sympy`. In this example, we take a sum of [Gaussians](https://en.wikipedia.org/wiki/Gaussian_function) plus some [Poisson](https://en.wikipedia.org/wiki/Poisson_distribution) distribution.

In [ ]:
def gaussian(x: sp.Symbol, mu: sp.Symbol, sigma: sp.Symbol) -> sp.Expr:
    return sp.exp(-(((x - mu) / sigma) ** 2) / 2)


def poisson(x: sp.Symbol, k: sp.Symbol) -> sp.Expr:
    return x**k * sp.exp(-x) / sp.factorial(k)

In [ ]:
x, mu, sigma = sp.symbols("x mu sigma")
k = sp.Symbol("k", integer=True)
lam = sp.Symbol("lambda", positive=True)
style = "<style>#output-body{display:flex; flex-direction: row;}</style>"
display(
    gaussian(x, mu, sigma),
    poisson(lam, k),
)

In [ ]:
x, a, b, c, mu1, mu2, sigma1, sigma2 = sp.symbols("x (a:c) mu_(:2) sigma_(:2)")
expression_1d = (
    a * gaussian(x, mu1, sigma1)
    + b * gaussian(x, mu2, sigma2)
    + c * poisson(x, k=2)
)
expression_1d

The expression above consists of a number of {class}`~sympy.core.symbol.Symbol`s that we want to identify as **parameters** (that we want to optimize with regard to a certain data sample) and **variables** (in which the data sample is expressed). Let's say $x$ is the variable and that the rest of the {class}`~sympy.core.symbol.Symbol`s are the parameters.

Here, we'll pick some default values for the parameter and use them to plot the model with regard to the variable $x$ (see {meth}`~sympy.core.basic.Basic.subs`). The default values are used later on as well when we generate data.

```{margin}
We use {attr}`~sympy.core.basic.Basic.args` here to extract the components of the sum that forms this expression. See {doc}`sympy:tutorial/manipulation`.
```

In [ ]:
parameter_defaults = {
    a: 0.15,
    b: 0.05,
    c: 0.3,
    mu1: 1.0,
    sigma1: 0.3,
    mu2: 2.7,
    sigma2: 0.5,
}
x_range = (x, 0, 5)
substituted_expr_1d = expression_1d.subs(parameter_defaults)
p1 = sp.plot(substituted_expr_1d, x_range, show=False, line_color="red")
p2 = sp.plot(*substituted_expr_1d.args, x_range, show=False, line_color="gray")
p2.append(p1[0])
p2.show()

## Convert to backend

So far, all we did was using {mod}`sympy` to symbolically formulate a mathematical expression. We now need to {func}`~sympy.utilities.lambdify.lambdify` that expression to some computational backend, so that we can efficiently generate data and/or optimize the parameters in the function to 'fit' the model to some data sample. TensorWaves can do this with the {func}`.create_parametrized_function` function:

In [ ]:
function_1d = create_parametrized_function(
    expression=expression_1d,
    parameters=parameter_defaults,
    backend="jax",
    use_cse=False,
)

:::{tip}

<!-- cspell:ignore subexpression -->
Here, we used `use_cse=False` in {func}`.create_parametrized_function`. Setting this argument to `True` (the default) causes {mod}`sympy` to search for common sub-expressions, which speeds up lambdification in large expressions and makes the lambdified source code more efficient. See also {func}`~sympy.simplify.cse_main.cse`.

:::

The resulting {class}`.ParametrizedBackendFunction` internally carries some source code that {mod}`numpy` understands. With {func}`.get_source_code`, we can see that it indeed looks similar to the expression that we formulated in {ref}`usage/basics:Formulate model`:

In [ ]:
from black import FileMode, format_str

from tensorwaves.function import get_source_code

src = get_source_code(function_1d)
src = format_str(src, mode=FileMode())
print(src)

The {class}`.ParametrizedBackendFunction` also carries the original default values for the {attr}`~.ParametrizedBackendFunction.parameters` that we defined earlier on.

In [ ]:
function_1d.parameters

The {meth}`.ParametrizedFunction.__call__` takes a {class}`dict` of variable names (here, `"x"` only) to the value(s) that should be used in their place.

In [ ]:
function_1d({"x": 0})

This is where we move to the data generation ― the input values are usually a list of values (expressed in the backend):

In [ ]:
rng = np.random.default_rng()
x_values = np.linspace(0, 5, num=20)
y_values = function_1d({"x": x_values})
y_values

In [ ]:
plt.scatter(x_values, y_values)
plt.gca().set_xlabel("$x$")
plt.gca().set_ylabel("$f(x)$");

## Generate data

So, we now have a function $f$ of $x$ expressed in some computational backend. This function is to describe a distribution over $x$. In the real world, $x$ is an observable from a process you measure. But sometimes, it's useful to generate a 'toy' data sample for your model function as well, to try it out.

### Hit & miss

The challenge is to generate values of $x$ with a density that is proportional to the value of the function evaluated at that point. To do this, we use a **hit & miss** approach:

1. Generate a random value for $x$ within the domain $(x_\mathrm{min}, x_\mathrm{max})$ on which you want to generate the data sample.
2. Generate a random value $y$ between $0$ and the maximum value $y_\mathrm{max}$ of the function over the domain of $x$.
3. Check if $y$ lies below $f(x)$ ("hit") or above ("miss").
4. If there is a "hit", accept this value of $x$ and add it to the data sample.

We keep performing this until the sample of $x$ values contains the desired number of events.

In [ ]:
x_domain = np.linspace(0, 5, num=200)
y_values = function_1d({"x": x_domain})
fig = plt.figure(figsize=(8, 5))
plt.plot(x_domain, y_values)
plt.gca().set_xlabel("$x$")
plt.gca().set_ylabel("$f(x)$")

x_min = x_range[1]
x_max = x_range[2]
y_max = 0.21
x_value = 1.5

line_segment = [[0, 0], [0, y_max]]
plt.plot(*line_segment, color="black")
plt.text(
    -0.22,
    y_max / 2 * 0.5,
    "uniform sample $(0, y_{max})$",
    rotation="vertical",
)
plt.axhline(y=y_max, linestyle="dotted", color="black")
plt.text(
    x_min + 0.1,
    y_max - 0.01,
    "$y_{max}$",
)

line_segment = [[x_min, x_max], [0, 0]]
plt.plot(*line_segment, color="black")
plt.text(
    (x_max - x_min) / 2 - 0.22,
    0.005,
    R"uniform sample $(x_\mathrm{min}, x_\mathrm{max})$",
)
plt.scatter(x_value, function_1d({"x": x_value}))
plt.axvline(x=x_value, linestyle="dotted")


def draw_y_hit(x_random, y_random):
    y_value = function_1d({"x": x_random})
    color = "green" if y_random < y_value else "red"
    text = "hit" if y_random < y_value else "miss"
    plt.scatter(0, y_random, color=color)
    plt.arrow(
        x=0,
        y=y_random,
        dx=x_random,
        dy=0,
        head_length=0.15,
        length_includes_head=True,
        color=color,
        linestyle="dotted",
    )
    plt.text(x_value + 0.05, y_random, text)


draw_y_hit(x_random=x_value, y_random=0.05)
draw_y_hit(x_random=x_value, y_random=0.17)

There is one problem though: _how to determine $y_\mathrm{max}$?_ In this example, we can just read off the value of $y_\mathrm{max}$, or even compute it analytically from the original {class}`sympy.Expr <sympy.core.expr.Expr>`. This is not the case generally though, so we need to apply a trick.

Since we are generating **uniformly distributed** random values values of $x$ and computing their $f(x)$ values, we can keep track of which values of $f(x)$ is the highest. Starting with $y_\mathrm{max} = 0$ we just set $y_\mathrm{max} = f(x)$ once $f(x) > y_\mathrm{max}$ and _completely restart_ the generate loop. Eventually, some value of $x$ will lie near the absolute maximum of $f$ and the data generation will happily continue until the requested number of events has been reached.

```{warning}
There are two caveats:
1. The domain sample (here: the uniformly distributed values of $x$) has to be large in order for the data sample to accurately describe the original function.
2. The the function displays narrow structures, like some sharp global maximum containing $y_\mathrm{max}$, changes are smaller that the value of $x$ will lie within this peak. The domain sample will therefore have to be even larger. It will also take longer before $y_\mathrm{max}$ is found.
```

### Domain distribution

First of all, we need to randomly generate values of $x$. In this simple, 1-dimensional example, we could just use a random generator like {class}`numpy.random.Generator` feed its output to the {meth}`.ParametrizedFunction.__call__`. Generally, though, we want to cover $n$-dimensional cases. The class {class}`.NumpyDomainGenerator` allows us to generate such a **uniform** distribution for each variable within a certain range. It requires a {class}`.RealNumberGenerator` (here we use {class}`.NumpyUniformRNG`) and it also requires us to define boundaries for each variable in the resulting {obj}`.DataSample`.

In [ ]:
from tensorwaves.data import NumpyDomainGenerator, NumpyUniformRNG

rng = NumpyUniformRNG(seed=0)
domain_generator = NumpyDomainGenerator(boundaries={"x": (0, 5)})
domain = domain_generator.generate(1_000_000, rng)

:::{tip}

Set a `seed` in the {class}`.RealNumberGenerator` if you want to generate **deterministic** data sample. If you leave it unspecified, you get an **indeterministic** data sample.

:::

When we feed the sample generated domain sample to the {class}`.ParametrizedBackendFunction` and use it its output values as weights to the histogram of the uniform domain sample, we see that the domain nicely produces a distribution as expected from the {ref}`model we defined <usage/basics:Formulate model>`:

In [ ]:
plt.hist(
    domain["x"],
    bins=200,
    density=True,
    alpha=0.5,
    label="uniform",
)
plt.hist(
    domain["x"],
    weights=np.array(function_1d(domain)),
    bins=200,
    alpha=0.5,
    density=True,
    label="weighted with $f$",
)
plt.legend();

```{note}
In PWA, the sample on which we perform hit-and-miss is not uniform, because the available space is limited by the masses of the initial and final state (phase space). See {class}`.TFPhaseSpaceGenerator` and {ref}`amplitude-analysis:Step 2: Generate data`.
```

### Intensity distribution

With a {ref}`usage/basics:Domain distribution` in hand, we can work out an implementation for the {ref}`usage/basics:Hit & miss` approach. The {class}`.IntensityDistributionGenerator` class helps us to do this:

In [ ]:
from tensorwaves.data import IntensityDistributionGenerator

data_generator = IntensityDistributionGenerator(domain_generator, function_1d)
data = data_generator.generate(1_000_000, rng)

And indeed, it results in the correct distribution!

In [ ]:
plt.hist(data["x"], bins=200);

## Optimize the model

For the rest, the procedure is really just the same as that sketched in {ref}`compwa-step-3`.

We tweak the parameters a bit, then use {meth}`.ParametrizedBackendFunction.update_parameters` to change the function...

In [ ]:
initial_parameters = {
    "a": 0.2,
    "b": 0.1,
    "c": 0.2,
    "mu_0": 0.9,
    "sigma_0": 0.4,
    "sigma_1": 0.4,
}
function_1d.update_parameters(initial_parameters)

...compare what this looks like compared to the data...

In [ ]:
plt.hist(data["x"], bins=200, density=True)
plt.hist(
    domain["x"],
    weights=np.array(function_1d(domain)),
    bins=200,
    histtype="step",
    color="red",
    density=True,
);

...define an {class}`.Estimator` and choose {obj}`jax <jax.jit>` as backend...

In [ ]:
estimator = UnbinnedNLL(function_1d, data, domain, backend="jax")

...optimize with {class}`.Minuit2` (the `callback` argument is optional---see {ref}`usage/basics:Callbacks`).

In [ ]:
minuit2 = Minuit2(
    callback=CallbackList(
        [
            CSVSummary("traceback-1D.csv"),
            YAMLSummary("fit-result-1D.yaml"),
            TFSummary(),
        ]
    )
)
fit_result = minuit2.optimize(estimator, initial_parameters)
fit_result

In [ ]:
assert fit_result.minimum_valid

:::{tip}

For complicated expressions, the fit can be made faster with {func}`.create_cached_function`. See {doc}`/usage/caching`.

:::

And again, we have a look at the resulting fit, as well as what happened during the optimization.

In [ ]:
optimized_parameters = fit_result.parameter_values
function_1d.update_parameters(optimized_parameters)

In [ ]:
plt.hist(data["x"], bins=200, density=True)
plt.hist(
    domain["x"],
    weights=np.array(function_1d(domain)),
    bins=200,
    histtype="step",
    color="red",
    density=True,
);

### Callbacks

The {class}`.Minuit2` optimizer above was constructed with {mod}`.callbacks`. Callbacks allow us to insert behavior into the fit procedure of the optimizer. In this example, we use {class}`.CallbackList` to stack some {class}`.Callback` classes: {class}`.CSVSummary`, {class}`.YAMLSummary`, and {class}`.TFSummary`.

 {class}`.YAMLSummary` writes the latest fit result to disk. It's a {class}`.Loadable` callable and can be used to **pick up a fit later on**, for instance if it was aborted.

In [ ]:
latest_parameters = YAMLSummary.load_latest_parameters("fit-result-1D.yaml")
latest_parameters

In [ ]:
optimizer = Minuit2()
optimizer.optimize(estimator, latest_parameters)

{class}`.CSVSummary` records the parameter values in each iteration and can be used to **analyze the fit process**:

In [ ]:
fit_traceback = pd.read_csv("traceback-1D.csv")
fig, (ax1, ax2) = plt.subplots(
    2, figsize=(7, 9), sharex=True, gridspec_kw={"height_ratios": [1, 2]}
)
fit_traceback.plot("function_call", "estimator_value", ax=ax1)
fit_traceback.plot("function_call", sorted(initial_parameters), ax=ax2)
fig.tight_layout()
ax2.set_xlabel("function call");

{class}`.TFSummary` provides a nice, **interactive representation of the fit process** and can be viewed with [TensorBoard](https://www.tensorflow.org/tensorboard/get_started) as follows:

````{tabbed} Terminal
```bash
tensorboard --logdir logs
```
````

````{tabbed} Python
```python
import tensorboard as tb

tb.notebook.list()  # View open TensorBoard instances
tb.notebook.start(args_string="--logdir logs")
```
See more info [here](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks#tensorboard_in_notebooks)
````

````{tabbed} Jupyter notebook
```ipython
%load_ext tensorboard
%tensorboard --logdir logs
```
See more info [here](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks#tensorboard_in_notebooks)
````

## Example in 2D

The idea illustrated above works for any number of dimensions. Let's create multiply the expression we had with some $\cos$ as a function of $y$:

In [ ]:
y, omega = sp.symbols("y omega")
expression_2d = expression_1d * sp.cos(y * omega) ** 2
expression_2d

In [ ]:
parameter_defaults[omega] = 0.5

In [ ]:
y_range = (y, -sp.pi, +sp.pi)
substituted_expr_2d = expression_2d.subs(parameter_defaults)
plot3d(substituted_expr_2d, x_range, y_range)

In [ ]:
function_2d = create_parametrized_function(
    expression=expression_2d,
    parameters=parameter_defaults,
    backend="jax",
)

### Generate 2D data

In [ ]:
boundaries = {"x": (0, 5), "y": (-np.pi, +np.pi)}
domain_generator_2d = NumpyDomainGenerator(boundaries)
data_generator_2d = IntensityDistributionGenerator(
    domain_generator_2d, function_2d
)
domain_2d = domain_generator_2d.generate(300_000, rng)
data_2d = data_generator_2d.generate(30_000, rng)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))
intensities = np.array(function_2d(domain_2d))
kwargs = {
    "weights": intensities,
    "bins": 100,
    "density": True,
}
axes[0].hist(domain_2d["x"], **kwargs)
axes[1].hist(domain_2d["y"], **kwargs)
axes[0].set_xlabel("$x$")
axes[1].set_xlabel("$y$")
axes[0].set_ylabel("$f(x, y)$")
axes[0].set_yticks([])
axes[1].set_yticks([])
fig.tight_layout()

### Perform fit with different optimizers

In [ ]:
initial_parameters = {
    "a": 0.1,
    "b": 0.1,
    "c": 0.2,
    "mu_0": 0.9,
    "omega": 0.35,
    "sigma_0": 0.4,
    "sigma_1": 0.4,
}
function_2d.update_parameters(initial_parameters)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9, 4), sharey=True, tight_layout=True)
axes[0].hist2d(**data_2d, bins=50)
axes[1].hist2d(**domain_2d, weights=function_2d(domain_2d), bins=50)
axes[0].set_xlabel("$x$")
axes[0].set_ylim([-3, +3])
axes[1].set_xlabel("$x$")
axes[0].set_ylabel("$y$")
axes[0].set_title("Data sample")
axes[1].set_title("Function with optimized parameters");

#### Minuit2

Commonly, one would construct a {class}`.Minuit2` instance and call its {meth}`~.Minuit2.optimize` method. For more advanced options, one could specify a small `minuit_modifier` protocol into the {class}`.Minuit2` constructor. In this example, we set the {attr}`~iminuit.Minuit.tol` attribute. For other options, see {class}`iminuit.Minuit`.

In [ ]:
def tweak_minuit(minuit) -> None:
    minuit.tol = 0.2

For the rest, the fit procedure goes just as in {ref}`usage:Optimize parameters`:

In [ ]:
estimator = UnbinnedNLL(function_2d, data_2d, domain_2d, backend="jax")
minuit2 = Minuit2(
    callback=CSVSummary("traceback.csv"),
    minuit_modifier=tweak_minuit,
)
fit_result = minuit2.optimize(estimator, initial_parameters)
fit_result

Note that further information about the internal {class}`iminuit.Minuit` optimizer is available through {attr}`.FitResult.specifics`, e.g. computing the {meth}`~iminuit.Minuit.hesse` afterwards:

In [ ]:
fit_result.specifics.hesse()

#### Scipy

In [ ]:
scipy = ScipyMinimizer(
    method="Nelder-Mead",
    callback=CSVSummary("traceback-scipy.csv"),
)
fit_result = scipy.optimize(estimator, initial_parameters)
fit_result

:::{warning}

Scipy does not provide error values for the optimized parameters.

:::

### Analyze fit process

If we update the parameters in the {class}`.ParametrizedFunction` with the optimized parameter values found by the {class}`.Optimizer`, we can compare the data distribution with the function.

In [ ]:
optimized_parameters = fit_result.parameter_values
function_2d.update_parameters(optimized_parameters)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9, 4), sharey=True, tight_layout=True)
fig.suptitle("Final fit result")
axes[0].hist2d(**data_2d, bins=50)
axes[1].hist2d(**domain_2d, weights=function_2d(domain_2d), bins=50)
axes[0].set_xlabel("$x$")
axes[0].set_ylim([-3, +3])
axes[1].set_xlabel("$x$")
axes[0].set_ylabel("$y$")
axes[0].set_title("Data sample")
axes[1].set_title("Function with optimized parameters");

In addition, the {ref}`callbacks <usage/basics:Callbacks>` allow us to inspect how the parameter values evolved during the fit with the {class}`.ScipyMinimizer` and {class}`.Minuit2` optimizers:

In [ ]:
minuit_traceback = pd.read_csv("traceback.csv")
scipy_traceback = pd.read_csv("traceback-scipy.csv")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(
    ncols=2,
    nrows=2,
    figsize=(10, 9),
    gridspec_kw={"height_ratios": [1, 3]},
)
fig.suptitle("Evolution of the parameter values during the fit")
ax1.set_title("Minuit2")
ax2.set_title("Scipy (Nelder-Mead)")
ax1.get_shared_x_axes().join(ax1, ax3)
ax2.get_shared_x_axes().join(ax2, ax4)
ax1.get_shared_y_axes().join(ax1, ax2)
ax3.get_shared_y_axes().join(ax3, ax4)
ax2.set_ylim(
    1.02 * scipy_traceback["estimator_value"].min(),
    0.98 * scipy_traceback["estimator_value"].max(),
)
minuit_traceback.plot("function_call", "estimator_value", ax=ax1, legend=False)
scipy_traceback.plot("function_call", "estimator_value", ax=ax2)
minuit_traceback.plot(
    "function_call", initial_parameters, ax=ax3, legend=False
)
scipy_traceback.plot(
    "function_call", initial_parameters, ax=ax4, legend=True
).legend(loc="upper right")
fig.tight_layout()
ax2.set_xlabel("function call");